In [1]:
import findspark
findspark.init('/usr/local/spark')

In [2]:
import datetime
import geopy.distance
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.window import Window
from pyspark.sql.functions import struct, udf, col, desc, row_number, min, max, lag
from pyspark.sql.types import *

In [3]:
sc = SparkContext(master="local[*]")
spark = SparkSession.builder.appName("Assignment 2 Wuwei Zhang").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
spark.sparkContext.getConf().getAll()

22/01/24 17:34:26 WARN Utils: Your hostname, ZdeMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.10.0.76 instead (on interface en0)
22/01/24 17:34:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/24 17:34:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[('spark.sql.warehouse.dir',
  'file:/Users/zwill/PycharmProjects/bigData_analysis/pyspark_analysis/spark-warehouse'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.host', '10.10.0.76'),
 ('spark.driver.port', '62335'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1643045667524'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.startTime', '1643045666821'),
 ('spark.app.name', 'pyspark-shell')]

In [5]:
dataset = [('A', 2, 1, 70), ('A', 3, 5, 80), ('C', 5, 3, 90), ('C', 5, 4, 100), ('E', 6, 5, 110)]

In [9]:
# d
from operator import add
rdd = sc.parallelize(dataset)
# Select the first and last element of each tuple
x = rdd.map(lambda rd: (rd[0], rd[3]))
x= x.reduceByKey(add)
x.collect()

[('C', 190), ('A', 150), ('E', 110)]

In [7]:
 # e
from operator import add
rdd = sc.parallelize(dataset)
# Select the first,second and last element of each tuple and group by the first element and second element
x2 = rdd.map(lambda rd: ((rd[0], rd[1]), rd[3]))
# reduce by the first and second element
x2 = x2.reduceByKey(add)
x2.collect()

[(('C', 5), 190), (('E', 6), 110), (('A', 2), 70), (('A', 3), 80)]

In [8]:
# f
# Output a x[2] when the greatest number of x[3]
rdd = sc.parallelize(dataset)
x3 = rdd.map(lambda rd: (rd[2], rd[3]))
x3 = x3.reduceByKey(add)
x3.max()

(5, 190)

In [10]:
schema = StructType([
    StructField("UserId", IntegerType(), True),
    StructField("Latitude", DoubleType(), True),
    StructField("Longitude", DoubleType(), True),
    StructField("AllZero", IntegerType(), True),
    StructField("Altitude", DoubleType(), True),
    StructField("Timestamp", DoubleType(), True),
    StructField("Date", StringType(), True),
    StructField("Time", StringType(), True),
])

df = spark.read.csv('dataset.txt', header=True, schema=schema)
# Timestamp is sufficient to re-calculate the date and time
df = df.drop('Date', 'Time')
df.printSchema()

root
 |-- UserId: integer (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- AllZero: integer (nullable = true)
 |-- Altitude: double (nullable = true)
 |-- Timestamp: double (nullable = true)



In [11]:
# Time difference between Beijing and UTC
time_diff = 8 * 3600
# the difference between Epoch & Unix Timestamp and current time stamp base
interval = (datetime.datetime(1970, 1, 1) - datetime.datetime(1899, 12, 30)).total_seconds()
# change the timestamp to Unix Timestamp + 8 hours
df = df.withColumn('Timestamp', col('Timestamp') * 24 * 3600 + time_diff - interval)

to_date = udf(lambda x: datetime.datetime.fromtimestamp(x.Timestamp, datetime.timezone.utc).strftime('%Y-%m-%d'), StringType())
to_time = udf(lambda x: datetime.datetime.fromtimestamp(x.Timestamp, datetime.timezone.utc).strftime('%H:%M:%S'), StringType())

df = df.withColumn("Date",to_date(struct([df[x] for x in df.columns])))
df = df.withColumn("Time",to_time(struct([df[x] for x in df.columns])))

# Combine the latitude and longitude to a single column
df = df.withColumn("Coordinate",struct(df.Latitude, df.Longitude)).drop('Latitude', 'Longitude')

In [12]:
df.show(5)

+------+-------+----------------+--------------------+----------+--------+--------------------+
|UserId|AllZero|        Altitude|           Timestamp|      Date|    Time|          Coordinate|
+------+-------+----------------+--------------------+----------+--------+--------------------+
|   100|      0|480.287355643045|1.3119722519999962E9|2011-07-29|20:44:11|{39.974408918, 11...|
|   100|      0|480.121151574803|1.3119722529999986E9|2011-07-29|20:44:12|{39.974397078, 11...|
|   100|      0|478.499455380577|1.3119722550000029E9|2011-07-29|20:44:15|{39.973982524, 11...|
|   100|      0|479.176988188976|1.3119722559999962E9|2011-07-29|20:44:15|{39.973943291, 11...|
|   100|      0|479.129432414698|1.3119722569999986E9|2011-07-29|20:44:16|{39.973937148, 11...|
+------+-------+----------------+--------------------+----------+--------+--------------------+
only showing top 5 rows



In [13]:
'''
Calculate for each person, on how many days was the data recorded for them (count any day with
at least one data point).
'''
date_df = df.select('UserId','Date').drop_duplicates()
date_count = date_df.groupBy('UserID').count().sort(desc('count'))

In [14]:
date_count.show(5)

+------+-----+
|UserID|count|
+------+-----+
|   128|  910|
|   126|  178|
|   104|  117|
|   115|  116|
|   112|  109|
+------+-----+
only showing top 5 rows



In [9]:
'''
Calculate for each person, on how many days there were more than 100 data points recorded for them (count any day with at least 100 data points). Output all user IDs and the corresponding value of this measure.
'''
date_duplicate = df.select('UserId','Date').groupBy('UserID','Date').count().filter(col('count') >= 100)
df_user_date = date_duplicate.select('UserId').groupBy('UserID').count().sort(desc('count'))
df_user_date.show(df_user_date.count())

+------+-----+
|UserID|count|
+------+-----+
|   128|  873|
|   126|  171|
|   115|  100|
|   112|   98|
|   104|   86|
|   125|   48|
|   119|   45|
|   101|   31|
|   122|   24|
|   103|   22|
|   102|   21|
|   130|   20|
|   111|   20|
|   113|   19|
|   110|   16|
|   114|   15|
|   105|    8|
|   121|    7|
|   127|    7|
|   124|    6|
|   100|    5|
|   123|    4|
|   129|    3|
|   106|    3|
|   116|    3|
|   108|    2|
|   120|    2|
|   109|    2|
|   118|    2|
+------+-----+



In [15]:
'''
Calculate for each person, the highest altitude that they reached. Output the top 5 user ID according to this measure, its value and the day that was achieved.
'''
windowDept = Window.partitionBy("UserId").orderBy(col("Altitude").desc())
df_altitude = df.select('UserId','Altitude', 'Date')
df_altitude = df_altitude.withColumn("row",row_number().over(windowDept)).filter(col("row") == 1).drop("row").sort(desc('Altitude'))

In [16]:
df_altitude.show(5)

+------+----------------+----------+
|UserId|        Altitude|      Date|
+------+----------------+----------+
|   128|        107503.3|2009-11-02|
|   106|36581.3648293963|2007-10-09|
|   103|         25259.2|2008-09-12|
|   101|         24806.4|2008-03-28|
|   126|         19432.4|2008-06-22|
+------+----------------+----------+
only showing top 5 rows



In [12]:
'''
Calculate for each person, the timespan of the observation, i.e., the difference between the highest timestamp of his/her observation and the lowest one. Output the top 5 user ID according to this measure and its value.
'''
df_timestamp_extrema = df.select(
  'UserId', 'Timestamp',
).groupBy(
  'UserId'
).agg(
  min('Timestamp').alias('Timestamp_min'),
  max('Timestamp').alias('Timestamp_max'),
)

time_diff = udf(lambda x: x.Timestamp_max - x.Timestamp_min, DoubleType())
df_timespan = df_timestamp_extrema.withColumn('TimeSpan', time_diff(struct([df_timestamp_extrema[x] for x in df_timestamp_extrema.columns]))).sort(desc('TimeSpan'))
df_timespan = df_timespan.drop('Timestamp_min', 'Timestamp_max')

In [13]:
df_timespan.show(5)

+------+--------------------+
|UserId|            TimeSpan|
+------+--------------------+
|   128|        1.23231834E8|
|   114| 8.327625900000238E7|
|   111|          7.247087E7|
|   115| 4.377784700000143E7|
|   126|2.8151853999998093E7|
+------+--------------------+
only showing top 5 rows



In [8]:
'''
Calculate for each person, the distance travelled by them each day. For each user output the (earliest) day they travelled the most. Also, output the total distance travelled by all users on all days.
'''
df_distance = df.select('UserId','Coordinate','Date', 'Timestamp')
# calculate the distance between two points
def get_distance_between_two_points(row):
    coord1 = row.Coordinate
    coord2 = row.lag_Coordinate
    if coord1 is None or coord2 is None:
        return 0
    else:
        return  geopy.distance.geodesic(coord1, coord2).km

# lag the coordinate column
same_day_window = Window.partitionBy("UserID", "Date").orderBy("Timestamp")
df_distance = df_distance.withColumn('lag_Coordinate', lag(df_distance['Coordinate']).over(same_day_window))

distance_udf = udf(get_distance_between_two_points, DoubleType())
# calculate the distance travelled by each user each coordinate change
df_distance = df_distance.withColumn('DistanceByCoordinate', distance_udf(struct([df_distance[x] for x in df_distance.columns])))
# calculate the total distance travelled by each user each day
df_distance_agg = df_distance.groupBy('UserID', 'Date').sum('DistanceByCoordinate').sort(desc('sum(DistanceByCoordinate)')).withColumnRenamed('sum(DistanceByCoordinate)', "DistanceByDay")

# calculate the max daily distance travelled by each user
distance_user_window = Window.partitionBy("UserID").orderBy(col('DistanceByDay').desc())
df_distance_user = df_distance_agg.select('UserId','DistanceByDay','Date').withColumn("row",row_number().over(distance_user_window)).filter(col("row") == 1).drop("row").sort(desc('DistanceByDay'))

In [9]:
print('Step 6')

Step 6


In [15]:
df_distance_user.sort('UserId').show(df_distance_user.count())

+------+------------------+----------+
|UserId|     DistanceByDay|      Date|
+------+------------------+----------+
|   100|12.173128258276241|2011-08-09|
|   101|228.45342441436495|2007-12-23|
|   102|29.849437524843076|2011-12-31|
|   103|194.34882725105257|2008-09-12|
|   104|112.41388625202974|2008-09-11|
|   105| 58.97419690548178|2007-10-06|
|   106|252.92611172528123|2007-10-09|
|   107| 8.570857323698148|2007-10-07|
|   108|165.43560669850862|2007-10-04|
|   109|35.484262239303455|2007-12-01|
|   110| 89.48598039196608|2008-01-19|
|   111|2459.9727492396887|2007-09-05|
|   112|118.84986296601792|2008-07-01|
|   113|37.250444802782006|2010-06-03|
|   114| 43.13695902871845|2010-05-29|
|   115| 851.9804449907956|2008-09-13|
|   116|3.3736070308458106|2011-08-03|
|   117|15.522764119627311|2007-06-29|
|   118| 395.8065623103833|2007-05-20|
|   119|139.38895930121691|2008-08-29|
|   120| 436.0508695845096|2009-09-19|
|   121|129.46860882496455|2009-10-09|
|   122|157.7818980277891

In [24]:
# sum the total distance travelled by all users
df_distance.select('DistanceByCoordinate').groupBy().sum('DistanceByCoordinate').show()

+-------------------------+
|sum(DistanceByCoordinate)|
+-------------------------+
|         136320.404922086|
+-------------------------+



In [10]:
df_distance_agg.select('DistanceByDay').groupBy().sum('DistanceByDay').show()

+------------------+
|sum(DistanceByDay)|
+------------------+
|136320.40492208552|
+------------------+

